In [60]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import seaborn as sns

### Very rough plan

data:
- standardize data types for each predictor
- dummy variables for categorical
- for NA values: either 0 or mean values of category or eliminate player?
- graphs
    - CarAV by position, CarAV versus all factors, counts of categorical factors, means of numerical factors



analysis:
- linear reg for entire model, offense/defense, by position 
    - approximate value (CAV) as outcome 
- refit dropping factors with low p-values
- normalize features so coefficient values are meaningful
- lasso regression
- classification into groups based on cut off of CAV for labels such as pro-bowl level, starter, bench, bad
- cluster?


Use best model on most recent draft class to predict success

### Data analysis and visualizations

In [55]:
#read in predictor data (ncaa, combine)
predictor2 = pd.read_csv("/Users/kirk.hachigian/Desktop/21SpringClasses/4740/project/predictor_data.csv")

#limit data for 2000 (start of data) to 2016
predictor = predictor2[predictor2['year']<=2016]

#read in outcome data (nfl)
outcome2 = pd.read_csv("/Users/kirk.hachigian/Desktop/21SpringClasses/4740/project/outcome_data.csv")
outcome2 = outcome2.rename(columns={'Player':'player'})

#select career approximate value metric for each player
outcome3 = outcome2[['player','CarAV']]

#add approximate value metric for each player in predictor dataset
df = pd.merge(predictor,outcome3,on='player')
df

,url,pick,team,year,pos,player,college,missing_combine_vals,height_inches,weight,...,rushing.rec.yards,rushing.receptions,rushing.rush.att,rushing.rush.td,rushing.rush.yds,rushing.scrim.plays,rushing.scrim.tds,rushing.scrim.yds,rushing.seasons,CarAV
0,https://www.sports-reference.com/cfb/players/a...,67.0,JAX,2015,OG,A.J. Cann,South Carolina,4,75,313,...,0,0,0,0,0,0,0,0,0,6.0
1,https://www.sports-reference.com/cfb/players/a...,105.0,DET,2007,CB,A.J. Davis,North Carolina State,4,70,193,...,0,0,0,0,0,0,0,0,0,0.0
2,https://www.sports-reference.com/cfb/players/a...,300.0,NaN,2010,OLB,A.J. Edds,Iowa,0,76,246,...,0,0,0,0,0,0,0,0,0,0.0
3,https://www.sports-reference.com/cfb/players/a...,4.0,CIN,2011,WR,A.J. Green,Georgia,0,76,211,...,0,0,0,0,0,0,0,0,0,53.0
4,https://www.sports-reference.com/cfb/players/a...,5.0,GNB,2006,OLB,A.J. Hawk,Ohio State,0,73,248,...,0,0,0,0,0,0,0,0,0,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3041,https://www.sports-reference.com/cfb/players/z...,300.0,NaN,2010,OG,Zane Beadles,Utah,4,76,310,...,0,0,0,0,0,0,0,0,0,42.0
3042,https://www.sports-reference.com/cfb/players/z...,97.0,TEN,2013,OLB,Zaviar Gooden,Missouri,0,73,234,...,0,0,0,0,0,0,0,0,0,3.0
3043,https://www.sports-reference.com/cfb/players/z...,144.0,BUF,2012,OT,Zebrie Sanders,Florida State,0,78,320,...,0,0,0,0,0,0,0,0,0,0.0
3044,https://www.sports-reference.com/cfb/players/z...,139.0,SDG,2001,ILB,Zeke Moreno,Southern California,2,74,248,...,0,0,0,0,0,0,0,0,0,10.0


In [62]:
dfd = pd.get_dummies(df)
dfd

,pick,year,missing_combine_vals,height_inches,weight,forty,vertical,bench,broad,threecone,...,rushing.scrim.yds_848,rushing.scrim.yds_855,rushing.scrim.yds_88,rushing.scrim.yds_888,rushing.scrim.yds_909,rushing.scrim.yds_914,rushing.scrim.yds_951,rushing.scrim.yds_955,rushing.scrim.yds_980,rushing.scrim.yds_989
0,67.0,2015,4,75,313,5.47,NaN,26.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,105.0,2007,4,70,193,4.56,NaN,16.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,300.0,2010,0,76,246,4.62,33.0,16.0,117.0,7.19,...,0,0,0,0,0,0,0,0,0,0
3,4.0,2011,0,76,211,4.48,34.5,18.0,126.0,6.91,...,0,0,0,0,0,0,0,0,0,0
4,5.0,2006,0,73,248,4.59,40.0,24.0,115.0,6.82,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3041,300.0,2010,4,76,310,5.28,NaN,27.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3042,97.0,2013,0,73,234,4.47,34.0,27.0,131.0,6.71,...,0,0,0,0,0,0,0,0,0,0
3043,144.0,2012,0,78,320,5.33,27.0,28.0,100.0,8.15,...,0,0,0,0,0,0,0,0,0,0
3044,139.0,2001,2,74,248,4.76,NaN,24.0,NaN,7.10,...,0,0,0,0,0,0,0,0,0,0


### Initial analysis

In [45]:
#example linreg based off only 3 predictors: pick, height, weight
X = df[['pick', 'height_inches', 'weight']]
Y = df['CarAV']

#test/train split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [56]:
linreg = linear_model.LinearRegression()
scores = cross_validate(linreg, X_train, Y_train, scoring=('r2', 'neg_mean_squared_error'), cv=10, return_train_score=True)
model_final = linreg.fit(X_train,Y_train)

In [58]:
final_predict = model_final.predict(X_test)
print('Coefficients: \n', model_final.coef_)
print('Mean squared error: %.2f'
      % mean_squared_error(Y_test, final_predict))
print('Coefficient of determination: %.2f'
      % r2_score(Y_test, final_predict))

Coefficients: 
 [-0.06751759 -0.0483252   0.02537128]
Mean squared error: 327.58
Coefficient of determination: 0.12
